In [1]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

170508288/170498071 [==============================] - 4s 0us/step


In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 10),
 (10000, 32, 32, 3),
 (10000, 10),
 (10000, 32, 32, 3),
 (10000, 10))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def alexnet(activation):
    AlexNet = Sequential()

    AlexNet.add(Conv2D(filters=96, input_shape=(32, 32, 3), kernel_size=(11, 11), strides=(4, 4), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Flatten())
    AlexNet.add(Dense(4096, input_shape=(32, 32, 3)))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(1000))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(10))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('softmax'))

    return AlexNet

In [9]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = alexnet('selu')

batch_size = 128
epochs = 50

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 25s 71ms/step - loss: 1.8960 - accuracy: 0.3184 - val_loss: 1.5365 - val_accuracy: 0.4502
Epoch 2/50
313/313 [==============================] - 22s 71ms/step - loss: 1.5119 - accuracy: 0.4747 - val_loss: 1.4960 - val_accuracy: 0.4627
Epoch 3/50
313/313 [==============================] - 23s 72ms/step - loss: 1.3896 - accuracy: 0.5189 - val_loss: 1.4891 - val_accuracy: 0.4704
Epoch 4/50
313/313 [==============================] - 23s 74ms/step - loss: 1.2912 - accuracy: 0.5602 - val_loss: 1.4892 - val_accuracy: 0.4745
Epoch 5/50
313/313 [==============================] - 21s 69ms/step - loss: 1.2155 - accuracy: 0.5881 - val_loss: 1.2959 - val_accuracy: 0.5525
Epoch 6/50
313/313 [==============================] - 22s 69ms/step - loss: 1.1487 - accuracy: 0.6147 - val_loss: 1.1501 - val_accuracy: 0.5976
Epoch 7/50
313/313 [==============================] - 23s 75ms/step - loss: 1.0877 - accuracy: 0.6359 - val_loss: 1.4652 - val_accuracy:

In [10]:
print(history.history)
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test,axis=1)

print(y_pred.shape)
print(y_true.shape)

print(np.sum(y_pred == y_true) / y_pred.shape[0])

{'loss': [1.7349658012390137, 1.4862544536590576, 1.3679304122924805, 1.281173825263977, 1.2060599327087402, 1.1437054872512817, 1.0901273488998413, 1.0425764322280884, 0.9998759627342224, 0.9603866934776306, 0.9148302674293518, 0.8888217806816101, 0.8440009951591492, 0.8138599991798401, 0.7842612266540527, 0.7520037889480591, 0.7176622152328491, 0.6903333067893982, 0.6561369895935059, 0.6252542734146118, 0.5992385149002075, 0.5690187811851501, 0.5446377992630005, 0.514930784702301, 0.4900778830051422, 0.46530991792678833, 0.4397892653942108, 0.41581249237060547, 0.39445292949676514, 0.3706732392311096, 0.34919193387031555, 0.3282613456249237, 0.3048030138015747, 0.2865076959133148, 0.26617518067359924, 0.24677348136901855, 0.23263974487781525, 0.22052380442619324, 0.2081049531698227, 0.19694693386554718, 0.177347794175148, 0.16180773079395294, 0.15038080513477325, 0.13958679139614105, 0.1315694898366928, 0.12010334432125092, 0.11323820799589157, 0.11003167927265167, 0.0989624187350273